# Hydrogen Orbitals Part 2

In this notebook, we'll look at a few more ways to plot data, and will work on refactoring our `h_orbital` code into a python class to make our code more modular and reusable. This will also make it easy for us to transform into Cartesian coordinates instead of spherical coordinates, and allow us to plot orbitals from different atoms on the same graph. Along the way, we'll discuss good coding practices.

## Creating the HAtom class

Classes are data structures that contain both variables and functions. When using a class in your code, you create an **object** which is an instance of the class. The class definition tells what the object is made from and how it is built, and the object itself is what interacts with the rest of your code. Using classes and objects in your code has several benefits:

- **Data organization**: The variables associated with an object are bundled together. As we'll see, if we want to plot mutliple atoms, we might want to keep track of each atom's center position and quantum numbrers. Instead of keeping track of everything in a master list, we can instead create each atom as an object that keeps track of its own data. It also keeps related code together so that when there is a problem, you know where to look to fix it.
- **Reusability**: Once you have created a class that you like, you can import that class into other code that you write. Many of the python data structures you've used (strings, figures, axes) are classes.
- **Flexibility**: Classes can be "extended" by creating a subclass that inherits all of the variables and functions from the parent class. This makes it easy to add new functionality without having to rewrite everything from scratch.

We'll start by building a basic class for the hydrogen atom. If you have never built a python class, have a look at this [basic tutorial](http://openbookproject.net/thinkcs/python/english3e/classes_and_objects_I.html) before reading the code below. Particularly pay attention to the meanings of `__init__` and `self`!

In [ ]:
import numpy as np
import scipy as sp
import scipy.special as sps

class HAtom:
    """ Class for representing a hydrogen atom
    
    Here we can provide examples of how to use the class, and list the
    important functions a user of the class may want to access. Since this
    is a tutorial, we will omit the details here and show it in code.
    """
    
    def __init__(self, n : int, l : int, m : int) -> None:
        """ Constructor: sets the n, l, and m quantum numbers """
        
        self.n = n
        self.l = l
        self.m = m
        
    def orbital(self, r : np.ndarray, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the $r$, $\theta$, $\phi$ grid
        
        Note: Here we could add a detailed description of the how the orbital
        is represented, noting that the angular convention we're using is opposite
        that of sp.special.sph_harm.
        
        Parameters
        ----------
        r : np.ndarray
            $r$ values, in atomic units
        theta : np.ndarray
            colatitudinal angle in radians, in the range [0,pi)
        phi : np.ndarray
            azimuthal angle in radians, in the range [0,2pi]
        
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at r, theta, phi
        """
        
        pf = ( (2./self.n)**3. * sps.factorial(self.n-self.l-1) / (2. * self.n * sps.factorial(self.n+self.l)) )**0.5
        radial = np.exp(-r/2.) * r**self.l * sps.eval_genlaguerre(self.n-self.l-1, 2*self.l+1, r)
        angular = sps.sph_harm(self.m, self.l, phi, theta)
        return pf * radial * angular
    
    def __repr__(self) -> str:
        return f'{type(self).__name__}(n={self.n}, l={self.l}, m={self.m})'

The HAtom class consists of three functions:

- `__init__` is the function that initializes an instance of the class. When we create an object, we need to provide the arguments to this function, such as `h1 = HAtom(1,0,0)`.
- `orbital` is the same function as `h_orbital` from the previous notebook. However, we do not need to provide the `n`, `l`, and `m` values when we call this function because they are already stored when we created the object.
- `__repr__` is a special python function that tells the interpreter how to represent the object when it is entered as a command or in the print function. See the next code block for an example.

All of the information in quotes are [docstrings](https://www.python.org/dev/peps/pep-0257/). When creating a class, it is good practice to describe what the class is used for, and to document all of the functions and attributes that a class user needs to know about. Writing detailed comments in your code is **always** a good idea, especially when the code is complicated. Good comments will save you time in the long run, and will make it much easier for people to work on your code (especially your future self!).

Finally, we've included [type hints](https://www.python.org/dev/peps/pep-0484/) into the function signature, which lets the user know what kinds of input the functions expect and what kind of output they produce. The arguments to `__init__` are all integers, and it does not return anything, while the `orbital` function expects arrays and returns an array. Python as a language uses [Duck Typing](https://en.wikipedia.org/wiki/Duck_typing), so it doesn't enforce that the parameters you pass to a function be of the types specifiec by the type hints. They are mostly used to inform the end user of how you, as the programmer, intended for the function to work, even if python allows it to be used in other ways that you did not intend. There are also some external tools that can be used to analyze your code and warn you if a parameter you pass to a function doesn't match the function's type hints, and that can be useful to find bugs in large projects.

To create an `HAtom` object we need to provide the arguments to the `__init__` function like this.

In [ ]:
h1 = HAtom(2,1,0)
print(h1)

Here we have created an `HAtom` named `h1` and specified its quantum numbers. When calling `print`, python looks in the class for a `__str__` function, and if it doesn't find one, it uses `__repr__`. We implemented only `__repr__` so that the interpreter prints the string when it encounters an `HAtom` object:

In [ ]:
h1

## Using Cartesian coordinates in the HAtom class

The plots we made in the previous notebook are fine, but it is sometimes difficult to visualize how a slice in $(r,\phi)$ space maps onto normal $(x,y,z)$ space. It would be more convenient to make our graphs in Cartesian coordinates, while still using spherical coordinates to calculate the orbital. We'll write a function called `to_spherical` that converts Cartesian coordinates to spherical coordinates, and then write a new function in the `HAtom` class that makes use of it.

Normally, to modify the class you would just return to the earlier cell, make changes, and then re-run it. But for instructional purposes, in this notebook we'll make new versions of the class as we go. An important point to note about class definitions in Jupyter notebooks is that if you modify a class, any objects of that class that you have already created still refer to the **old version** of the class. You must recreate the object to use the updated class definition, as shown below.

Finally, just for ease of future use, we will include all of the import statements that are needed for the class in the same cell as the definition.

In [ ]:
import numpy as np
import scipy as sp
import scipy.special as sps
from typing import Tuple

def to_spherical(x : np.ndarray, y : np.ndarray, z : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Converts the Cartesian coordinates x,y,z to r,theta,phi

    This function expects x, y, and z to be broadcastable 3D arrays,
    such as those generated by np.meshgrid. The output arrays are
    always dense 3D grids, and correspond to r, theta, and phi

    Parameters
    ----------
    x : np.ndarray
        Cartesian x values, in a.u.
    x : np.ndarray
        Cartesian y values, in a.u.
    x : np.ndarray
        Cartesian z values, in a.u.

    Returns
    -------
    np.ndarray
        3D grid of r values
    np.ndarray
        3D grid of theta values
    np.ndarray
        3D grid of phi values
    """

    r = ( x**2. + y**2. + z**2. )**0.5
    theta = np.arccos(z/r)
    phi = np.arctan(y/x)

    return r, theta, phi

class HAtom:
    """ Class for representing a hydrogen atom
    
    Here we can provide examples of how to use the class, and list the
    important functions a user of the class may want to access. Since this
    is a tutorial, we will omit the details here and show it in code.
    """
    
    def __init__(self, n : int, l : int, m : int):
        """ Constructor: sets the n, l, and m quantum numbers """
        
        self.n = n
        self.l = l
        self.m = m
    
    def orbital_xyz(self, x : np.ndarray, y : np.ndarray, z : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the x,y,z grid
        
        The x, y, and z values are converted to spherical coordinates,
        then evaluated
        
        Parameters
        ----------
        x : np.ndarray
            Cartesian x values, in a.u.
        x : np.ndarray
            Cartesian y values, in a.u.
        x : np.ndarray
            Cartesian z values, in a.u.
            
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at x, y, z
        """
        
        r, theta, phi = to_spherical(x, y, z)
        return self.orbital(r, theta, phi)
        
    def orbital(self, r : np.ndarray, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the $r$, $\theta$, $\phi$ grid
        
        Note: Here we could add a detailed description of the how the orbital
        is represented, noting that the angular convention we're using is opposite
        that of sp.special.sph_harm.
        
        Parameters
        ----------
        r : np.ndarray
            $r$ values, in atomic units
        theta : np.ndarray
            colatitudinal angle in radians, in the range [0,pi)
        phi : np.ndarray
            azimuthal angle in radians, in the range [0,2pi]
        
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at r, theta, phi
        """
        
        pf = ( (2./self.n)**3. * sps.factorial(self.n-self.l-1) / (2. * self.n * sps.factorial(self.n+self.l)) )**0.5
        radial = np.exp(-r/2.) * r**self.l * sps.eval_genlaguerre(self.n-self.l-1, 2*self.l+1, r)
        angular = sps.sph_harm(self.m, self.l, phi, theta)
        return pf * radial * angular
    
    def __repr__(self):
        return f'{type(self).__name__}(n={self.n}, l={self.l}, m={self.m})'

We've added the `to_spherical` function into the global namespace because it doesn't depend on anything in the `HAtom` class, then added `HAtom.orbital_xyz` that makes use of that function to do the conversion. Let's try it out. Note that since we have redefined the class, currently our `h1` object is still pointing to the old class definition and does not have an `orbital_xyz`function. We need to recreate the object now that the class definition has changed.

In [ ]:
xyz = np.linspace(-10,10,51)
x, y, z = np.meshgrid(xyz,xyz,xyz,sparse=True)

# This throws an exception because orbital_xyz wasn't in the class
# definition when h1 was created
print(h1)
try:
    psi = h1.orbital_xyz(x,y,z)
except Exception as e:
    print(f'{type(e).__name__}: {e}')
    
h1 = HAtom(2,1,1)
print(h1)

try:
    psi = h1.orbital_xyz(x,y,z)
except Exception as e:
    print(f'{type(e).__name__}: {e}')
    

The code completed without raising an exception, but there are several warnings about invalid values and divide by zero! If you already know why, good. Here we'll take a moment to figure out how to track this down. The RuntimeWarning is generated by `numpy`, which has an error handler that is invoked when problems with floating-point operations happen. A warning like this allows the program to continue executing, but it prints the warning message to alert you to the fact that the results may not match what you expect.

You can view and change numpy's error handling with [`numpy.geterr`](https://numpy.org/doc/stable/reference/generated/numpy.geterr.html) and [`numpy.seterr`](https://numpy.org/doc/stable/reference/generated/numpy.seterr.html). We can force numpy to halt the code when it encounters divide by zero by calling `np.seterr(all='raise')`. This tells us exactly where the problem is.

In [ ]:
print(np.geterr())

old_settings = np.seterr(all='raise')
try:
    psi = h1.orbital_xyz(x,y,z)
finally:
    np.seterr(**old_settings) # restore original settings

As you may have supected, we have a problem at the origin. When `x=0`, `y=0`, and `z=0`, there are issues with the coordinate system because $\theta$ and $\phi$ are ill-defined, and the formulas we've used involve division by `r` and `x`. Let's take a look at some plots of the orbital we've tried to make using [`Axes.imshow`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html), which is designed to make 2D plots with evenly spaced data along the two axes. Note in the code below the use of [`pyplot.subplots`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) to make the figure and a 2D array of `Axes` objects, which is then flattened and zipped with the functions and labels to create the plots in a loop. You can change `z_index` and rerun the cell to take a different cross section through the orbital, and explore how the [`imshow` interpolation methods](https://matplotlib.org/gallery/images_contours_and_fields/interpolation_methods.html) change the images.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm

fig, axes = plt.subplots(2,2, figsize=(12,12))

funcs = [np.real,np.imag,np.absolute,np.angle]
labels = ['Real','Imag','Mag','Phase']
z_index = 25

for ax,f,label in zip(axes.flatten(),funcs,labels):
    d = f(psi[:,:,z_index])
    
    #this line ensures that the color scales are symmetric around 0
    #When used with the RdBu color map, this ensures that blue is positive,
    #red is negative, and white is 0.
    #Becasue there are NaNs where there was division by 0, use nanmax
    norm = matplotlib.cm.colors.Normalize(vmax=np.nanmax(np.absolute(d)), vmin=-np.nanmax(np.absolute(d)))
    
    
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation='none',extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    cb.set_label(label)
    ax.set_title(f'z = {xyz[z_index]}')
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    
fig.tight_layout()

This visualization lets us see that there are is a problem not only at $r=0$, but also at $x=0$: the sign of the imaginary part is abruptly changing, which we can also see in the phase plot. Note that the apparent discontinuity at $y=0$ in the phase plot corresponds to a change from $-\pi \to \pi$, which is the normal expected behavior for a function that is $2\pi$-periodic.

To fix our spherical coordinate conversion, we should just set $\theta = 0$ when $r = 0$, and we need to be careful at $x=0$ when calculating $\phi$. For the first problem, you might think that we can calculate `r`, then use `if np.isclose(r,0.0): theta = 0.0`. However, that won't work because `theta` needs to be an array, not a single number. Instead, we can use the [`np.where`] function to accomplish the same thing, as shown in the code below.

The $\phi$ problem is so common that a special version of the arctangent function has been created to deal with it: the two-argument tangent function [`np.arctan2`](https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html). With these corrections in place, the new `to_spherical` function becomes the following.

In [ ]:
def to_spherical(x : np.ndarray, y : np.ndarray, z : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Converts the Cartesian coordinates x,y,z to r,theta,phi

    This function expects x, y, and z to be broadcastable 3D arrays,
    such as those generated by np.meshgrid. The output arrays are
    always dense 3D grids, and correspond to r, theta, and phi

    Parameters
    ----------
    x : np.ndarray
        Cartesian x values, in a.u.
    x : np.ndarray
        Cartesian y values, in a.u.
    x : np.ndarray
        Cartesian z values, in a.u.

    Returns
    -------
    np.ndarray
        3D grid of r values
    np.ndarray
        3D grid of theta values
    np.ndarray
        3D grid of phi values
    """

    r = ( x**2. + y**2. + z**2. )**0.5
    theta = np.where(np.isclose(r,0.0),np.zeros_like(r),np.arccos(z/r))
    phi = np.arctan2(y,x)

    return r, theta, phi


psi = h1.orbital_xyz(x,y,z)

You might be surprised to see that we're still getting a warning message about an invalid value in the divide function. This is due to the way that the `np.where` function works: first both `np.zeros_like` and `np.arccos` are evaluated at **all** values of `r` to build 2 arrays (we'll call them `a` and `b`). Then, `np.where` evaluates the condition `np.isclose(r,0.0)` for each value of `r`, and chooses the value from array `a` if the condition is true or array `b` if it is false. Because `np.arccos` is evaluated at `r=0`, there is a divide by zero, so a warning is brought up. Since we know that this is unavoidable, we can suppress the warning ising the python [`warnings`](https://docs.python.org/3/library/warnings.html) module.

In [ ]:
import warnings

def to_spherical(x : np.ndarray, y : np.ndarray, z : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Converts the Cartesian coordinates x,y,z to r,theta,phi

    This function expects x, y, and z to be broadcastable 3D arrays,
    such as those generated by np.meshgrid. The output arrays are
    always dense 3D grids, and correspond to r, theta, and phi

    Parameters
    ----------
    x : np.ndarray
        Cartesian x values, in a.u.
    x : np.ndarray
        Cartesian y values, in a.u.
    x : np.ndarray
        Cartesian z values, in a.u.

    Returns
    -------
    np.ndarray
        3D grid of r values
    np.ndarray
        3D grid of theta values
    np.ndarray
        3D grid of phi values
    """

    r = ( x**2. + y**2. + z**2. )**0.5
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        theta = np.where(np.isclose(r,0.0),np.zeros_like(r),np.arccos(z/r))
    phi = np.arctan2(y,x)

    return r, theta, phi


psi = h1.orbital_xyz(x,y,z)

Now the warning is suppressed. To make sure that this works properly, lets replot the orbital using the code above. (here you could just re-execute the earlier cell, but to make the notebook sequential that cell is copied below and edited to remove the code that handled NaNs). We can see that our earlier discontinuities are gone. The magnitude plot correctly shows that the wavefunction is 0 at the origin, and we see that the phase varies smoothly with $\phi$, which is the angle measured in the $xy$ plane.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm

fig, axes = plt.subplots(2,2, figsize=(12,12))

funcs = [np.real,np.imag,np.absolute,np.angle]
labels = ['Real','Imag','Mag','Phase']
z_index = 25

for ax,f,label in zip(axes.flatten(),funcs,labels):
    d = f(psi[:,:,z_index])
    norm = matplotlib.cm.colors.Normalize(vmax=abs(d).max(), vmin=-abs(d).max())
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation='none',extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    cb.set_label(label)
    ax.set_title(f'z = {xyz[z_index]}')
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    
fig.tight_layout()

## Changing the atom's position and orbital

So far we've assumed that the atom is located at the origin of our coordinate system. Let's change that so that we can visualize orbitals from multiple hydrogen atoms at different places, and be able to look at different orbitals from the same atom more easily. This will highlight a key advantage of using classes.

We can add `x0`, `y0`, and `z0` variables to the `HAtom` class so that each atom keeps track of its own location. Then, when we convert `x`, `y`, and `z` to spherical coordinates, we need to subtract the atom's location coordinate from the Cartesian coordinate, like `x_ = (x-x0)`. This means that the `to_spherical` function needs to know `x0`, `y0`, and `z0`. We can either modify the function so that it takes 6 arguments or, better, make the function a part of the class itself, which we'll do here.

In [ ]:
import numpy as np
import scipy as sp
import scipy.special as sps
from typing import Tuple
import warnings

class HAtom:
    """ Class for representing a hydrogen atom
    
    Here we can provide examples of how to use the class, and list the
    important functions a user of the class may want to access. Since this
    is a tutorial, we will omit the details here and show it in code.
    """
    
    def __init__(self, n : int, l : int, m : int, x0 : float = 0.0, y0 : float = 0.0, z0 : float = 0.0):
        """ Constructor: sets the quantum numbers and position
        
        Quantum numbers are required. If position is not specified,
        defaults to (0.0,0.0,0.0)
        """
        
        self.set_orbital(n,l,m)
        self.set_position(x0,y0,z0)
        
    def set_orbital(self, n : int, l : int, m : int) -> None:
        """ Sets quantum numbers """
        
        self.n = n
        self.l = l
        self.m = m
        
    def set_position(self, x0 : float, y0 : float, z0 : float) -> None:
        """ Sets position in a.u"""
        
        self.x0 = x0
        self.y0 = y0
        self.z0 = z0
        
    def to_spherical(self, x : np.ndarray, y : np.ndarray, z : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Converts the Cartesian coordinates x,y,z to r,theta,phi

        This function expects x, y, and z to be broadcastable 3D arrays,
        such as those generated by np.meshgrid. The output arrays are
        always dense 3D grids, and correspond to r, theta, and phi. The
        atom's center position is taken into account.

        Parameters
        ----------
        x : np.ndarray
            Cartesian x values, in a.u.
        x : np.ndarray
            Cartesian y values, in a.u.
        x : np.ndarray
            Cartesian z values, in a.u.

        Returns
        -------
        np.ndarray
            3D grid of r values
        np.ndarray
            3D grid of theta values
        np.ndarray
            3D grid of phi values
        """
        
        x_ = x-self.x0
        y_ = y-self.y0
        z_ = z-self.z0
        
        r = ( x_**2. + y_**2. + z_**2. )**0.5
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            theta = np.where(np.isclose(r,0.0),np.zeros_like(r),np.arccos(z_/r))
        phi = np.arctan2(y_,x_)

        return r, theta, phi
    
    def orbital_xyz(self, x : np.ndarray, y : np.ndarray, z : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the x,y,z grid
        
        The x, y, and z values are converted to spherical coordinates,
        then evaluated
        
        Parameters
        ----------
        x : np.ndarray
            Cartesian x values, in a.u.
        x : np.ndarray
            Cartesian y values, in a.u.
        x : np.ndarray
            Cartesian z values, in a.u.
            
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at x, y, z
        """
        
        r, theta, phi = self.to_spherical(x, y, z)
        return self.orbital(r, theta, phi)
        
    def orbital(self, r : np.ndarray, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the $r$, $\theta$, $\phi$ grid
        
        Note: Here we could add a detailed description of the how the orbital
        is represented, noting that the angular convention we're using is opposite
        that of sp.special.sph_harm.
        
        Parameters
        ----------
        r : np.ndarray
            $r$ values, in atomic units
        theta : np.ndarray
            colatitudinal angle in radians, in the range [0,pi)
        phi : np.ndarray
            azimuthal angle in radians, in the range [0,2pi]
        
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at r, theta, phi
        """
        
        pf = ( (2./self.n)**3. * sps.factorial(self.n-self.l-1) / (2. * self.n * sps.factorial(self.n+self.l)) )**0.5
        radial = np.exp(-r/2.) * r**self.l * sps.eval_genlaguerre(self.n-self.l-1, 2*self.l+1, r)
        
        return pf * radial * self.angular(theta,phi)
    
    def angular(self, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the angular part of the hydrogen wavefuntion"""
        
        return sps.sph_harm(self.m, self.l, phi, theta)
        
    
    def __repr__(self):
        return f'{type(self).__name__}(n={self.n}, l={self.l}, m={self.m}) at (x,y,z) = ({self.x0},{self.y0},{self.z0})'
    
h1 = HAtom(1,0,0)

print(h1)

h1.set_orbital(2,1,1)
h1.set_position(3.0,-1.5,0.0)
psi = h1.orbital_xyz(x,y,z)

print(h1)

We've made a few modifications to the class:

- `to_spherical` is now part of the class. Its function signature now has the first argument `self`, which refers to the instance of the class, and allows the function to access the object's `x0`, `y0`, and `z0` values.
- `__init__` now allows optional argupemts that specify `x0`, `y0`, and `z0`
- A new `set_orbital` function makes it easy to change all the quantum numbers at once, instead of having to do `h1.n = 2`, `h1.l = 1`, `h1.m = m`
- A new `set_position` function does the same for `x0`, `y0`, and `z0`
- The `orbital_xyz` function calls `self.to_spherical` instead of the `to_spherical` function in the global namespace
- Moved the calculation of the angular part of the orbital to its own function named `angular`, which will be useful in our next step.
- The `__repr__` function now prints the atom's position in addition to its quantum numbers

We can verify that the class is working by plotting the wavefunction on the same grid as before.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm

fig, axes = plt.subplots(2,2, figsize=(12,12))

funcs = [np.real,np.imag,np.absolute,np.angle]
labels = ['Real','Imag','Mag','Phase']
z_index = 25

for ax,f,label in zip(axes.flatten(),funcs,labels):
    d = f(psi[:,:,z_index])
    norm = matplotlib.cm.colors.Normalize(vmax=abs(d).max(), vmin=-abs(d).max())
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation='none',extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    cb.set_label(label)
    ax.set_title(f'z = {xyz[z_index]}')
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    
fig.tight_layout()

## Real forms of the orbitals

We now have the orbitals in Cartesian space, but they're still complex. The normal forms of the orbitals that we use in chemistry are converted into real numbers by manipulating the angular portion as shown in the table below. The resultant real orbitals are completely equivalent to the complex ones.

| m &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Real Form |
| --- | --- |
| $m < 0$ | $(-1)^m \sqrt{2}\, \text{Im}\left(Y_{\ell}^{|m|}\right) $ |
| $m = 0$ | $Y_\ell^0 $ |
| $m > 0$ | $(-1)^m\sqrt{2}\, \text{Re}\left(Y_\ell^{\phantom{|}m}\right) $ |

We'll add this by creating a subclass of `HAtom` that instead calculates the real form of the wavefunction. Just so that the code is all together, the `HAtom` class is copied again in the following cell.

In [ ]:
import numpy as np
import scipy as sp
import scipy.special as sps
from typing import Tuple
import warnings

class HAtom:
    """ Class for representing a hydrogen atom
    
    Here we can provide examples of how to use the class, and list the
    important functions a user of the class may want to access. Since this
    is a tutorial, we will omit the details here and show it in code.
    """
    
    def __init__(self, n : int, l : int, m : int, x0 : float = 0.0, y0 : float = 0.0, z0 : float = 0.0):
        """ Constructor: sets the quantum numbers and position
        
        Quantum numbers are required. If position is not specified,
        defaults to (0.0,0.0,0.0)
        """
        
        self.set_orbital(n,l,m)
        self.set_position(x0,y0,z0)
        
    def set_orbital(self, n : int, l : int, m : int) -> None:
        """ Sets quantum numbers """
        
        self.n = n
        self.l = l
        self.m = m
        
    def set_position(self, x0 : float, y0 : float, z0 : float) -> None:
        """ Sets position in a.u"""
        
        self.x0 = x0
        self.y0 = y0
        self.z0 = z0
        
    def to_spherical(self, x : np.ndarray, y : np.ndarray, z : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Converts the Cartesian coordinates x,y,z to r,theta,phi

        This function expects x, y, and z to be broadcastable 3D arrays,
        such as those generated by np.meshgrid. The output arrays are
        always dense 3D grids, and correspond to r, theta, and phi. The
        atom's center position is taken into account.

        Parameters
        ----------
        x : np.ndarray
            Cartesian x values, in a.u.
        x : np.ndarray
            Cartesian y values, in a.u.
        x : np.ndarray
            Cartesian z values, in a.u.

        Returns
        -------
        np.ndarray
            3D grid of r values
        np.ndarray
            3D grid of theta values
        np.ndarray
            3D grid of phi values
        """
        
        x_ = x-self.x0
        y_ = y-self.y0
        z_ = z-self.z0
        
        r = ( x_**2. + y_**2. + z_**2. )**0.5
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            theta = np.where(np.isclose(r,0.0),np.zeros_like(r),np.arccos(z_/r))
        phi = np.arctan2(y_,x_)

        return r, theta, phi
    
    def orbital_xyz(self, x : np.ndarray, y : np.ndarray, z : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the x,y,z grid
        
        The x, y, and z values are converted to spherical coordinates,
        then evaluated
        
        Parameters
        ----------
        x : np.ndarray
            Cartesian x values, in a.u.
        x : np.ndarray
            Cartesian y values, in a.u.
        x : np.ndarray
            Cartesian z values, in a.u.
            
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at x, y, z
        """
        
        r, theta, phi = self.to_spherical(x, y, z)
        return self.orbital(r, theta, phi)
        
    def orbital(self, r : np.ndarray, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the hydrogen orbital on the $r$, $\theta$, $\phi$ grid
        
        Note: Here we could add a detailed description of the how the orbital
        is represented, noting that the angular convention we're using is opposite
        that of sp.special.sph_harm.
        
        Parameters
        ----------
        r : np.ndarray
            $r$ values, in atomic units
        theta : np.ndarray
            colatitudinal angle in radians, in the range [0,pi)
        phi : np.ndarray
            azimuthal angle in radians, in the range [0,2pi]
        
        Returns
        -------
        np.ndarray
            3D array containing hydrogen atomic orbital evaluated at r, theta, phi
        """
        
        pf = ( (2./self.n)**3. * sps.factorial(self.n-self.l-1) / (2. * self.n * sps.factorial(self.n+self.l)) )**0.5
        radial = np.exp(-r/2.) * r**self.l * sps.eval_genlaguerre(self.n-self.l-1, 2*self.l+1, r)
        
        return pf * radial * self.angular(theta,phi)
    
    def angular(self, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """Calculates the angular part of the hydrogen wavefuntion"""
        
        return sps.sph_harm(self.m, self.l, phi, theta)
        
    
    def __repr__(self):
        return f'{type(self).__name__}(n={self.n}, l={self.l}, m={self.m}) at (x,y,z) = ({self.x0},{self.y0},{self.z0})'
    
class HAtomReal(HAtom):
    """ HAtom that calculates real orbitals """
    
    def angular(self, theta : np.ndarray, phi : np.ndarray) -> np.ndarray:
        """ Overrides HAtom.angular to calculate real spherical harmonics """
        
        if self.m > 0:
            return (-1)**self.m * np.sqrt(2.) * np.real(sps.sph_harm(self.m, self.l, phi, theta))
        elif self.m < 0:
            return (-1)**self.m * np.sqrt(2.) * np.imag(sps.sph_harm(-self.m, self.l, phi, theta))
        else:
            return np.real(super().angular(theta,phi))
    
    
h1 = HAtomReal(2,1,1)
print(h1)
psi = h1.orbital_xyz(x,y,z)

The real forms of the orbitals are those you are familiar with from general chemistry. The code below generates a 3x3 grid of plots taking different cross sections of the $2p$ orbitals. Each column is one of the orbitals, and each row corresponds to a slice in either the $xy$, $xz$, or $yz$ planes. From the plots, you can see that $m=-1$ is now the $p_y$ orbital, $m=0$ is the $p_z$ orbital, and $m=1$ is the $p_x$ orbital. Feel free to make adjustments to customize the plots.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm

fig, axes = plt.subplots(3,3, figsize=(12,12))

index = 25
n = 2
interp = 'none'

#Note: when imshow gets a 2D array, it interprets the first axis as row data (i.e., Y axis)
#and the second as column data (X axis). Because we used the default meshgrid, our data
#are organized such that axis 0 = y, axis 1 = x, and axis 2 = z.
#When taking slices, the following images are made:
#[:,:,a] : y vs x, z = a
#[:,a,:] : y vs z, x = a
#[a,:,:] : x vs z, y = a

#if z vs z is desired, then take the transpose: [a,:,:].T, as done in the third loop below

print(f'x shape: {x.shape}, y shape: {y.shape}, z shape: {z.shape}')

for ax,m in zip(axes[0],[-1,0,1]):
    h1.set_orbital(n,1,m)
    d = h1.orbital_xyz(x,y,z)[:,:,index]
    cmax = max(abs(d).max(),1e-7)
    norm = matplotlib.cm.colors.Normalize(vmax=cmax, vmin=-cmax)
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation=interp,extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    ax.set_title(f'(${n},1,{m}$), z = ${xyz[index]}$ $a_0$')
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    ax.set_xticks([-10,-5,0,5,10])
    ax.set_yticks([-10,-5,0,5,10])
    

for ax,m in zip(axes[1],[-1,0,1]):
    h1.set_orbital(n,1,m)
    d = h1.orbital_xyz(x,y,z)[:,index,:]
    cmax = max(abs(d).max(),1e-7)
    norm = matplotlib.cm.colors.Normalize(vmax=cmax, vmin=-cmax)
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation=interp,extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    ax.set_title(f'(${n},1,{m}$), x = ${xyz[index]}$ $a_0$')
    ax.set_xlabel('z ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    ax.set_xticks([-10,-5,0,5,10])
    ax.set_yticks([-10,-5,0,5,10])
    
for ax,m in zip(axes[2],[-1,0,1]):
    h1.set_orbital(n,1,m)
    d = h1.orbital_xyz(x,y,z)[index,:,:].T #transpose to get z vs x
    cmax = max(abs(d).max(),1e-7)
    norm = matplotlib.cm.colors.Normalize(vmax=cmax, vmin=-cmax)
    im = ax.imshow(d,cmap='RdBu',origin='lower',norm=norm,
                   interpolation=interp,extent=[xyz[0],xyz[-1],xyz[0],xyz[-1]])
    cb = fig.colorbar(im,ax=ax)
    ax.set_title(f'(${n},1,{m}$), y = ${xyz[index]}$ $a_0$')
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('z ($a_0$)')
    ax.set_xticks([-10,-5,0,5,10])
    ax.set_yticks([-10,-5,0,5,10])
    
fig.tight_layout()

## Intro to 3D plotting

The `matplotlib` library has support for 3D plotting through the [`mplot3d` interface](https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html). It is primarily designed to generate 3D plots where $z = f(x,y)$; that is, a 3D visualization of 2D functions. It is not a complete 3D graphics/rendering solution. Our situation is that we have a true 3D function, since $\psi = f(x,y,z)$. Typical visualizations of atomic orbitals are made by plotting [isosurfaces](https://en.wikipedia.org/wiki/Isosurface). Unfortunately, there is no one-line `matplotlib` plotting mechanism that can find isosurfaces, so in order to make such a plot we have to calculate the isosurface ourselves. The details are beyond the scope of the course, but some example code is shown below.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import skimage.measure as skm

xyz = np.linspace(-10,10,51)
x,y,z = np.meshgrid(xyz,xyz,xyz,sparse=True)


sp = abs(xyz[1]-xyz[0])
center = (xyz[-1]-xyz[0])/2

def add_isosurface_sym(ax,psi,level,spacing,alpha=0.5):
    for ll in [-level,level]:
        if float(ll) <= float(psi.min()) or float(ll) >= float(psi.max()):
            continue
        verts, faces, _, vals = skm.marching_cubes_lewiner(psi,level=ll,spacing=(spacing,spacing,spacing))
        verts -= np.asarray([[center,center,center]])
        mesh = Poly3DCollection(verts[faces])
        mesh.set_edgecolor('k')
        mesh.set_linewidth(0.05)
        if ll > 0:
            mesh.set_facecolor('#0000ff')
        else:
            mesh.set_facecolor('#ff0000')
        mesh.set_alpha(alpha)
        ax.add_collection3d(mesh)

fig = plt.figure(figsize=(13,4.2))
for plot in [1,2,3]:
    ax = fig.add_subplot(1,3,plot, projection='3d')

    if plot == 1:
        h1.set_orbital(3,1,0)
        title = '$3p\,z$'
    elif plot == 2:
        h1.set_orbital(3,2,0)
        title = '$3d\,z^2$'
    else:
        h1.set_orbital(4,3,0)
        title = '$4f\,z^3$'
        
    psi = h1.orbital_xyz(x,y,z)

    l = psi.max()
    for level in [0.25*l,0.5*l,0.75*l]:
        add_isosurface_sym(ax,psi,level,sp,0.3)

    ticks = [xyz[0], xyz[0]+(xyz[-1]-xyz[0])/4,xyz[0]+(xyz[-1]-xyz[0])/2,xyz[0]+3*(xyz[-1]-xyz[0])/4,xyz[-1]]
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    ax.set_zlim(-10,10)
    ax.set_xlabel('x ($a_0$)')
    ax.set_ylabel('y ($a_0$)')
    ax.set_zlabel('z ($a_0$)')
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_zticks(ticks)
    ax.set_title(title)
    
fig.tight_layout()



Although `matplotlib` has functions for controlling the angle and orentation of the figure, often it is more convenient to use an interactive plot to orient the figure as desired. This notebook is using the `inline` backend of matplotlib which only generates static figures, but the `notebook` backend (on some platforms) will generate interactive plots that can be zoomed, panned, and reoriented.

Another plotting utility that is even more convenient than `matplotlib` for isosurface plots is [`plotly`](https://plotly.com/python/), which generates interactive plots in the web browser. For graphs that you wish to be interactive, it is a good option. First you need to install `plotly`. Instructions are available at the link, but if you're using the Anaconda python distribution, you can just run `conda install plotly`. The code below sets up a basic isosurface plot using plotly; it's much simpler than the equivalent `matplotlib` code, and the interactive plot can be zoomed and panned.

In [ ]:
import plotly.graph_objects as go

xyz = np.linspace(-10,10,51)
x,y,z = np.meshgrid(xyz,xyz,xyz,sparse=False)

h1 = HAtomReal(2,1,0)
h1.set_position(0,0,0)
psi1 = h1.orbital_xyz(x,y,z)
psimax = abs(psi1).max()

fig= go.Figure(data=go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=psi1.flatten(),
    colorscale='RdBu',
    isomin=-.75*psimax,
    isomax=.75*psimax,
    surface_count=6,
    opacity=0.5,
    caps=dict(x_show=False,y_show=False,z_show=False)
))

fig.show()



One approximation of a chemical bond is just the sum (or differnce) of the orbitals on the atoms being bonded together. As a final example, we'll use `plotly` to visualize the overlap between a $2p_x$ orbital from one atom with the $3d_{xz}$ orbital of another atom. We'll do this by creating two `HAtomReal` objects. The first one is placed at $z=3$ and the second at $z=-3$. After calculating each orbital on the grid, we add the two together and plot. Here we'll also add markers showing the positions of the atoms, a line connecting them, and labels telling what the orbitals are.

In [ ]:
import plotly.graph_objects as go
import plotly.subplots

xyz = np.linspace(-10,10,51)
x,y,z = np.meshgrid(xyz,xyz,xyz,sparse=False)

h1 = HAtomReal(2,1,1)
h1.set_position(0,0,3)
psi1 = h1.orbital_xyz(x,y,z)


h2 = HAtomReal(3,2,1)
h2.set_position(0,0,-3)
psi2 = h2.orbital_xyz(x,y,z)

psi = psi1 + psi2
psimax = abs(psi).max()



fig= go.Figure(data=go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=psi.flatten(),
    colorscale='RdBu',
    isomin=-.75*psimax,
    isomax=.75*psimax,
    surface_count=6,
    opacity=0.5,
    caps=dict(x_show=False,y_show=False,z_show=False)
))

fig.add_trace(go.Scatter3d(
    x = [h1.x0,h2.x0],
    y = [h1.y0,h2.y0],
    z = [h1.z0,h2.z0],
    marker=dict(color='black'),
    line=dict(color='black',width=5)
))

fig.update_layout(
    autosize=False,
    width=800,
    height=700,
    margin=dict(l=20,r=20,b=20,t=20),
    title=dict(
        text="Orbital overlap",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    scene=dict(
        xaxis=dict(
            title_text=r'x (a<sub>0</sub>)'
        ),
        yaxis=dict(
            title_text=r'y (a<sub>0</sub>)'
        ),
        zaxis=dict(
            title_text=r'z (a<sub>0</sub>)'
        ),
        camera=dict(
            up=dict(x=0,y=0,z=1),
            eye=dict(x=0.2,y=2.5,z=0.2)
        ),
        annotations = [dict(
            showarrow=False,
            x = h1.x0,
            y = h1.y0,
            z = h1.z0,
            text = "2px",
            xanchor='left',
            xshift = 10,
            opacity=1.0
        ),dict(
            showarrow=False,
            x = h2.x0,
            y = h2.y0,
            z = h2.z0,
            text = "3dxz",
            xanchor='left',
            xshift = 10,
            opacity=1.0
    )])
)


fig.show()

# Key Takeaways

- [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) objects are the preferred way to represent numerical data, and `numpy` versions of math functions should be used because they allow for fast, [vectorized](https://numpy.org/doc/stable/glossary.html#term-vectorization) code and support [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).
- When working with floating point numbers, be aware of numerical stability and artifacts, especially when comparing floating point numbers or when dividing them (division by 0!)
- Classes and subclasses can be a useful way to make your code more modular, flexible, and reusable
- [`matplotlib`](https://matplotlib.org/) is a standard library for visualizing data. It works by creating a [`Figure`](https://matplotlib.org/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure) object and addint one or more [`Axes`](https://matplotlib.org/api/axes_api.html#matplotlib.axes.Axes) objects that represent individual graphs to display data. We looked at the following plotting methods
    - [`Axes.plot`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.plot.html) makes line graphs, good for looking at a 1D function or a set of (x,y) data points. A related method is [`Axes.scatter`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.scatter.html).
    - [`Axes.pcolormesh`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.pcolormesh.html) can show data of the form $f(x,y)$, and is useful when the data are on an irregualr grid. When the data are evenly spaced, [`Axes.imshow`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html) is preferred.
    - For 3D plotting, we looked briefly at the [`mplot3d` interface](https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html) and [`plotly`](https://plotly.com/python/), the latter of which is especially useful for making interactive plots on the web.

Finally, though we did not show it in these notebooks, figures made by `matplotlib` can be saved to an image file using [`Figure.savefig`](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure.savefig), which allows writing to a .png, .pdf, .eps., o .ps file (depending on the backend in use).

